# Import libraries

In [34]:
%pip install requests
%pip install pysqlite3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [79]:
import requests
import json
import time
from kafka import KafkaProducer

# Connection with an API

In [80]:
api_key= "RGAPI-945a005d-b5ab-4fbc-883a-c7f781727e4d" # API KEY que fica ativa somente por 24h (dia 01/11/2023)
url = "https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-name/LokoLoko00"

api_url = url + "?api_key=" + api_key
api_url

'https://br1.api.riotgames.com/lol/summoner/v4/summoners/by-name/LokoLoko00?api_key=RGAPI-945a005d-b5ab-4fbc-883a-c7f781727e4d'

In [81]:
response = requests.get(api_url)
response

<Response [200]>

In [82]:
response.json()

{'id': 'qgCJKD7lbLjZ1SxGckiPp87vdjaqnT13M6pzmf6wVSc4ExY',
 'accountId': '9C0i1eT680yVKF8_GzXjN7IxCg-F4iHYZaLZHsMDrKdCqb0qCxchERi8',
 'puuid': 'NPdm8EMy85kH4WLvCLF6ZMwvpezUG27Fqp2lvS4X2-KafoM9iRBwgk2FHI2FQ0mVxgY8kO_xflftMg',
 'name': 'LokoLoko00',
 'profileIconId': 665,
 'revisionDate': 1698645479000,
 'summonerLevel': 216}

# Envio do conteúdo

In [91]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')

In [92]:
if response.status_code == 200:
    data = response.json()

    for item, conteudo in data.items():
        print(f"Obtendo dados de perfil do LOL!")
        
        if conteudo:
            record = {
                item: conteudo,
            }

            json_record = json.dumps(record)
            
            print(json_record)

            producer.send('atividade_kafka', value=json_record.encode('utf-8'))

            print(f"Parte '{item}' dos dados de perfil, enviado para o Kafka.")
        else:
            print(f"Erro: Dados vazios, referente à {item}.")
    
    producer.flush()
    producer.close()

    print(f"Dados de perfil enviados com sucesso para o tópico do Kafka!")
else:
    print(f"Erro ao acessar a API da Riot. Código de status: {response.status_code}")

Obtendo dados de perfil do LOL!
{"id": "qgCJKD7lbLjZ1SxGckiPp87vdjaqnT13M6pzmf6wVSc4ExY"}
Parte 'id' dos dados de perfil, enviado para o Kafka.
Obtendo dados de perfil do LOL!
{"accountId": "9C0i1eT680yVKF8_GzXjN7IxCg-F4iHYZaLZHsMDrKdCqb0qCxchERi8"}
Parte 'accountId' dos dados de perfil, enviado para o Kafka.
Obtendo dados de perfil do LOL!
{"puuid": "NPdm8EMy85kH4WLvCLF6ZMwvpezUG27Fqp2lvS4X2-KafoM9iRBwgk2FHI2FQ0mVxgY8kO_xflftMg"}
Parte 'puuid' dos dados de perfil, enviado para o Kafka.
Obtendo dados de perfil do LOL!
{"name": "LokoLoko00"}
Parte 'name' dos dados de perfil, enviado para o Kafka.
Obtendo dados de perfil do LOL!
{"profileIconId": 665}
Parte 'profileIconId' dos dados de perfil, enviado para o Kafka.
Obtendo dados de perfil do LOL!
{"revisionDate": 1698645479000}
Parte 'revisionDate' dos dados de perfil, enviado para o Kafka.
Obtendo dados de perfil do LOL!
{"summonerLevel": 216}
Parte 'summonerLevel' dos dados de perfil, enviado para o Kafka.
Dados de perfil enviados com 